# RAG using local models



https://python.langchain.com/docs/use_cases/question_answering/local_retrieval_qa

In [1]:
!pip install bs4 tiktoken nltk langchain fake_useragent elasticsearch eland[pytorch]==8.10.1 --user


In [2]:
from getpass import getpass
from langchain.vectorstores import ElasticsearchStore


In [3]:
# ES_URL = input('Elasticsearch URL: ')
ES_URL = "https://127.0.0.1:9200" # input('Elasticsearch URL(ex:https://127.0.0.1:9200)')
ES_USER = "elastic" 
ES_USER_PASSWORD = "elastic" # getpass('elastic user PW: ')
CERT_PATH = "D:\es\8.10.4\kibana-8.10.4\data\ca_1698029751849.crt" # input('Elasticsearch pem 파일 경로: ')
# pem 생성 방법: https://cdax.ch/2022/02/20/elasticsearch-python-workshop-1-the-basics/

In [4]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_USER, ES_USER_PASSWORD),
    ca_certs=CERT_PATH,
    timeout=60
)

if client.indices.exists(index="workplace_index"):
    client.indices.delete(index="workplace_index")

In [5]:
# !eland_import_hub_model --url "https://127.0.0.1:9200" -u elastic -p elastic --ca-certs "D:\es\8.10.2\kibana-8.10.2\data\ca_1695716998259.crt" --hub-model-id "d:/Projects/es-lab-kr/s-core/models/distilbert-base-multilingual-cased" --task-type text_embedding --start --clear-previous

In [6]:
import os
cwd = os.getcwd()

try :
    os.mkdir(cwd + "/models") 
except:
    pass

In [7]:
os.chdir(cwd + "/models")

try :
    os.system("git clone https://huggingface.co/distilbert-base-multilingual-cased")
except:
    print('이미 모델이 존재합니다.')

os.chdir(cwd)

es_model_id = "sentence-transformers_distilbert-base-multilingual-cased"

In [8]:
import os
from pathlib import Path
from eland.ml import MLModel
from eland.ml.pytorch import PyTorchModel
from eland.common import es_version
from eland.ml.pytorch.transformers import TransformerModel

# 현재 경로 얻기
cwd = os.getcwd()
local_model_path = cwd + '/models/distilbert-base-multilingual-cased'

print(local_model_path)

# 모델 이름 및 작업 유형 설정
#tm = TransformerModel(local_model_path, "text_embedding")
es_cluster_version = es_version(client)
tm = TransformerModel(
    model_id=local_model_path, 
    task_type="text_embedding", 
    es_version=es_cluster_version
)

# Elasticsearch용 modelID를 설정
es_model_id = "sentence-transformers_distilbert-base-multilingual-cased"

es_model = MLModel(client, es_model_id)

if (es_model.exists_model() == False) :
    tmp_path = "models/sentence-transformers_distilbert-base-multilingual-cased"
    Path(tmp_path).mkdir(parents=True, exist_ok=True)
    model_path, config, vocab_path = tm.save(tmp_path)

    ptm = PyTorchModel(client, es_model_id)
    ptm.import_model(
        model_path=model_path,
        config_path=None,
        vocab_path=vocab_path,
        config=config
    ) 
    ptm.start()



d:\Projects\es-lab-kr\s-core/models/distilbert-base-multilingual-cased


No sentence-transformers model found with name d:\Projects\es-lab-kr\s-core/models/distilbert-base-multilingual-cased. Creating a new one with MEAN pooling.
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


In [9]:
from langchain.vectorstores import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

embeddings = ElasticsearchEmbeddings.from_es_connection(
    es_connection=client,
    model_id = es_model_id
)

vectorstore = ElasticsearchStore(
    es_connection=client, 
    embedding=embeddings, 
    index_name= "workplace_index"
)


In [10]:
# from urllib.request import urlopen
# from langchain.text_splitter import CharacterTextSplitter
# import json

# url = "./data.json"

# response = urlopen("https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/workplace-search/data/data.json")

# workplace_docs = json.loads(response.read())

# workplace_docs

In [11]:
# metadata = []
# content = []

# for doc in workplace_docs:
#   content.append(doc["content"])
#   metadata.append({
#       "name": doc["name"],
#       "summary": doc["summary"],
#       "rolePermissions":doc["rolePermissions"],
#   })

# text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
# docs = text_splitter.create_documents(content, metadatas=metadata)

# metadata
# docs

In [15]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.elastic.co/kr/blog/may-2023-launch-announcement")
loader.requests_kwargs = {'verify':False}

data = loader.load()

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0, 
    separator="\. ",
    length_function = len,
    is_separator_regex = True
)
docs = text_splitter.split_documents(data)

docs

c:\ProgramData\miniconda3\envs\chatbot\lib\site-packages\urllib3\connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'proxy.score'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
Created a chunk of size 2828, which is longer than the specified 1000


[Document(page_content='AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic BlogSkip to main contentPlatformPlatform솔루션솔루션고객고객리소스리소스요금제요금제설명서설명서Language pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin무료 체험판 사용 시작영업팀에 문의하기Elasticsearch PlatformElasticsearch Platform이 제공하는 Observability, Security, Search 솔루션Elastic 개요ELK Stack검색, 분석, 데이터 수집, 시각화를 간편하게KibanaElasticsearch통합ELK Stack 개요Elastic Cloud원하는 클라우드 서비스 제공자에서 Elastic을 사용하여 중요한 답을 찾아보세요.Cloud 개요파트너영업팀에 문의하기Observability앱과 인프라 가시성을 통합하여 문제를 사전에 해결합니다.로그 모니터링애플리케이션 성능 모니터링인프라 모니터링가상 모니터링실제 사용자 모니터링Universal ProfilingAIOpsOpenTelemetryObservability 개요Security대규모 사이버 위협으로부터 신속하게 보호하고 조사하고 대응하세요.지속적 모니터링위협 헌팅조사 및 사고 대응자동화된 위협 보호Security 개요Search모든 클라우드에 걸쳐 검색 결과를 가속화하고 맞춤형 검색을 강화하세요.생성형 AI검색 애플리케이션전자 상거래웹사이트Workplace Search고객 지원Search 개요산업별공공 부문금융 서비스통신의료기술소매 및 전자 상거래제조 및 자동차모든 산업 보기솔루션별고객에게 미래에 대비할 수 있는 유연성, 속도, 규모를 제공합니다.ObservabilitySecuritySearch고객 스포트라이트Cisco가 어떻게 AI를 통해 검색 경험을 혁신하는지 알아보세요.자세히 알아보기RWE가 어떻게

In [22]:
db = vectorstore.from_documents(
    docs, 
    embeddings, 
    es_connection=client,
    index_name= "workplace_index",
    distance_strategy="COSINE",
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(
        hybrid=True,
        query_model_id=es_model_id
    ),
    bulk_kwargs={
        "chunk_size": 500,
        "max_chunk_bytes": 800000
    }
)

In [23]:
def showResults(output):
  print("Total results: ", len(output))
  for index in range(len(output)):
    print(output[index])

In [24]:
query = "시맨틱 검색이란?"
results = db.similarity_search(query)

showResults(results)

Total results:  4
page_content='지금까지는 핵심 구성 요소인 RRF를 놓치고 있었습니다.\xa0 이제 애플리케이션 검색 요구 사항에 RRF를 포함하고 있으므로 벡터 및 텍스트 검색 기능을 결합할 수 있습니다.\xa0머신 러닝은 시맨틱 컨텍스트로 검색 결과의 정확도를 높이는 데 앞장서고 있지만, 비용, 복잡성 및 리소스 요구 사항이 너무 많아 개발자가 이를 효과적으로 구현하는 데 어려움을 겪고 있습니다\\. 개발자가 매우 정확한 AI 지원 검색을 구축하기 위해서는 전문 머신 러닝 또는 데이터 과학 팀의 지원이 필요한 경우가 일반적입니다\\. 이러한 팀은 적절한 모델을 선택하고, 도메인별 데이터 세트로 훈련하고, 데이터 및 해당 관계의 변경으로 인해 모델이 발전함에 따라 모델을 유지 관리하는 데 상당한 시간을 소비합니다.Go1이 확장 가능한 시맨틱 검색을 위해 어떻게 Elastic의 벡터 데이터베이스를 사용하는지 알아보세요.전문 팀의 지원을 받지 못하는 개발자도 시맨틱 검색을 구현할 수 있으며, 다른 대안에 필요한 노력과 전문 지식 없이 처음부터 AI 지원 검색 정확도의 이점을 누릴 수 있습니다\\. 오늘부터 모든 고객은 더 뛰어난 정확도와 현대적이고 스마트한 검색을 실현하는 데 도움이 되는 빌딩 블록을 갖추게 됩니다.사용해 보기이러한 기능과 그 외 다양한 기능에 대해 알아보세요.기존 Elastic Cloud 고객은 Elastic Cloud 콘솔에서 이 중 많은 기능을 바로 이용하실 수 있습니다\\. 클라우드에서 Elastic을 활용하고 있지 않으신가요? LLM 및 생성형 AI와 함께 Elasticsearch를 사용하는 방법을 확인해 보세요.이 게시물에 설명된 기능의 릴리즈 및 시기는 Elastic의 단독 재량에 따릅니다' metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticse

In [25]:
query = "How does the compensation work?"
results = db.similarity_search(query, k=10)

showResults(results)

Total results:  9
page_content='All rights reserved.Elastic, Elasticsearch 및 기타 관련 상표는 미국 및 기타 국가에서 Elasticsearch B.V.의 상표, 로고 또는 등록 상표입니다.Apache, Apache Lucene, Apache Hadoop, Hadoop, HDFS, 노란색 코끼리 로고는 미국 및/또는 기타 국가에서 Apache Software Foundation의 상표입니다.' metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic Blog', 'description': 'Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 모델을 사용하여 프라이빗 데이터 세트를 위한 생성형 AI(인공 지능) 솔루션을 지원합니다.', 'language': 'ko-kr'}
page_content='ESRE는 검색 분야를 선도하는 Elastic의 리더십과 2년에 걸친 머신 러닝 연구 및 개발을 기반으로 구축되었습니다\\. Elasticsearch Relevance Engine은 AI의 장점과 Elastic의 텍스트 검색을 결합합니다\\. ESRE는 개발자들에게 정교한 검색 알고리즘의 전체 제품군과 대규모 언어 모델(LLM)과 통합할 수 있는 기능을 제공합니다' metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic Blog', 'descrip

In [26]:
query = "생성형 AI가 생성하는 콘텐츠는 무엇일까?"
results = db.similarity_search(
    query
)

showResults(results)


Total results:  4
page_content='즉, 사용자의 의도를 이해하여 이전에는 볼 수 없었던 구체적인 응답을 제공할 수 있습니다.특히, 모든 검색의 발전은 새로운 기술과 변화하는 사용자 행동으로 인해 제기된 새로운 문제를 해결하는 동시에 더 나은 정확도를 제공합니다\\. 시맨틱 검색을 제공하기 위해 키워드 검색을 확장하든 비디오 및 이미지에 대한 새로운 검색 양식을 활성화하든, 새로운 기술에는 검색 사용자에게 더 나은 경험을 제공하기 위한 고유한 도구가 필요합니다\\. 마찬가지로 오늘날의 인공 지능 세계에서는 고객이 테스트한 검증된 기능을 갖춘 기술 스택에 구축된 확장성이 뛰어난 새로운 개발자 툴킷이 필요합니다.생성형 AI의 추진력과 더불어 ChatGPT와 같은 기술 채택이 증가하고, 대규모 언어 모델 기능에 대한 인식이 높아지면서 개발자는 애플리케이션을 개선할 수 있는 기술을 실험하고 싶어 합니다\\. Elasticsearch Relevance Engine은 생성형 AI 세계에서 새로운 기능의 시대를 열었으며 모든 개발자 팀이 즉시 사용할 수 있는 강력한 도구를 제공합니다.Elasticsearch Relevance Engine은 Elastic Cloud에서 지금 제공되고 있습니다\\. 최신 릴리즈의 모든 새로운 기능들은 유일한 호스트형 Elasticsearch 제품인 Elastic Cloud에서만 이용하실 수 있습니다' metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic Blog', 'description': 'Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 모델을 사용하여 프라이빗 데

In [27]:
query = "LLM의 역할을 알려줘?"
results = db.similarity_search(
    query
)

showResults(results)


Total results:  4
page_content='ESRE는 검색 분야를 선도하는 Elastic의 리더십과 2년에 걸친 머신 러닝 연구 및 개발을 기반으로 구축되었습니다\\. Elasticsearch Relevance Engine은 AI의 장점과 Elastic의 텍스트 검색을 결합합니다\\. ESRE는 개발자들에게 정교한 검색 알고리즘의 전체 제품군과 대규모 언어 모델(LLM)과 통합할 수 있는 기능을 제공합니다' metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic Blog', 'description': 'Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 모델을 사용하여 프라이빗 데이터 세트를 위한 생성형 AI(인공 지능) 솔루션을 지원합니다.', 'language': 'ko-kr'}
page_content='모든 개발 팀이 머신 러닝 모델을 훈련 및 유지관리하거나 확장성, 성능 및 속도에 대한 절충을 이해할 수 있는 리소스나 전문 지식을 보유하고 있는 것은 아니기 때문에 Elasticsearch Relevance Engine에는 다양한 도메인에 걸친 시맨틱 검색을 위해 구축된 검색 모델인 Elastic Learned Sparse Encoder도 포함되어 있습니다\\. 이 모델은 희소 임베딩을 기존의 키워드 기반 BM25 검색과 결합하여 하이브리드 검색을 위해 사용하기 쉬운 상호 순위 결합(RRF) 점수를 확보합니다\\. 개발자들은 ESRE를 통해 사용 첫날부터 머신 러닝 지원 정확도 및 하이브리드 검색 기술을 활용할 수 있습니다.개인정보 보호 및 보안: 데이터 개인정보 보호는 기업이 네트워크를 통해

In [28]:
!pip install llama-cpp-python

In [29]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [32]:
n_gpu_layers = 0  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path = cwd + "/models/Llama-2-ko-7B-chat-gguf-q8_0.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,

    # https://www.reddit.com/r/LocalLLaMA/comments/1343bgz/what_model_parameters_is_everyone_using/
    temperature=0.7,
    top_k=40,
    top_p=0.1,

    max_tokens=1024,
    verbose=True,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [33]:
llm("Large Language Model에 대해 설명해줘")

요.Large Language Model은 언어 모델의 크기를 크게 확장한 것입니다. 언어 모델은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니다.Large Language Model은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니다.Large Language Model은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니다.Large Language Model은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니다

'요.Large Language Model은 언어 모델의 크기를 크게 확장한 것입니다. 언어 모델은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니다.Large Language Model은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니다.Large Language Model은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니다.Large Language Model은 텍스트를 이해하고 해석하는 데 사용되는 인공 지능입니다. 예를 들어, 자연어 처리(NLP)는 텍스트에서 의미를 추출하여 인간과 상호 작용할 수 있는 컴퓨터 시스템을 만드는 것을 목표로 합니다.Large Language Model은 이러한 NLP의 한 유형입니다. 이 모델은 언어 모델의 크기를 크게 확장하여 더 많은 데이터를 학습하고 더 큰 데이터 세트에 대해 더 잘 작동하도록 설계되었습니

In [43]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Prompt
prompt = PromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)

# Chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Run
question = "Elasticsearch에 LLM을 적용하는 방법을 알려줘"
docs = vectorstore.similarity_search(question)
result = llm_chain(docs)

# Output
result

Llama.generate: prefix-match hit


], [Document(page_content='Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 모델을 사용하여 프라이빗 데이터 세트를 위한 생성형 AI(인공 지능) 솔루션을 지원합니다.', metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic Blog', 'description': 'Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 모델을 사용하여 프라이빗 데이터 세트를 위한 생성형 AI(인공 지능) 솔루션을 지원합니다.', 'language': 'ko-kr'}), Document(page_content='Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 모델을 사용하여 프라이빗 데이터 세트를 위한 생성형 AI(인공 지능) 솔루션을 지원합니다.', metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic Blog', 'description': 'Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 

{'docs': [Document(page_content='ESRE는 검색 분야를 선도하는 Elastic의 리더십과 2년에 걸친 머신 러닝 연구 및 개발을 기반으로 구축되었습니다\\. Elasticsearch Relevance Engine은 AI의 장점과 Elastic의 텍스트 검색을 결합합니다\\. ESRE는 개발자들에게 정교한 검색 알고리즘의 전체 제품군과 대규모 언어 모델(LLM)과 통합할 수 있는 기능을 제공합니다', metadata={'source': 'https://www.elastic.co/kr/blog/may-2023-launch-announcement', 'title': 'AI 검색 애플리케이션 구축을 위한 Elasticsearch Relevance Engine™ | Elastic Blog', 'description': 'Elasticsearch Relevance Engine™(ESRE)은 더 많은 검색 애플리케이션 개발자에게 더 높은 정확도를 제공하는 시맨틱 검색을 위한 벡터 데이터베이스와 머신 러닝 모델을 사용하여 프라이빗 데이터 세트를 위한 생성형 AI(인공 지능) 솔루션을 지원합니다.', 'language': 'ko-kr'}),
  Document(page_content='즉, 사용자의 의도를 이해하여 이전에는 볼 수 없었던 구체적인 응답을 제공할 수 있습니다.특히, 모든 검색의 발전은 새로운 기술과 변화하는 사용자 행동으로 인해 제기된 새로운 문제를 해결하는 동시에 더 나은 정확도를 제공합니다\\. 시맨틱 검색을 제공하기 위해 키워드 검색을 확장하든 비디오 및 이미지에 대한 새로운 검색 양식을 활성화하든, 새로운 기술에는 검색 사용자에게 더 나은 경험을 제공하기 위한 고유한 도구가 필요합니다\\. 마찬가지로 오늘날의 인공 지능 세계에서는 고객이 테스트한 검증된 기능을 갖춘 기술 스택에 구축된 확장성이 뛰어난 새로운 개발자 툴킷이 필요합니다.생성형 AI의 추진력과 더불어 ChatGPT와 같은 기술 채택이 증가하고, 대규모 언어 모델 기능

In [50]:
from langchain.chains import RetrievalQA

template = """[INST]<<SYS>> You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<</SYS>> 
Question: {question} 
Context: {context} 
Answer: [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["question", "context"])

# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)
question = "Elasticsearch에 LLM을 적용하는 방법을 알려줄 수 있니?"
result = qa_chain({"query": question})


Llama.generate: prefix-match hit


​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

KeyboardInterrupt: 